In [1]:
# general imports, basic globals

import io
import sys
import subprocess 
import copy
import pandas as pd
import numpy as np
import pyvista as pv

from gfq import *
from graph_viz import *
from graph_classes import *

# Define some colors, for the vertices and edges, used in default markings for L,W,C,V1,V2
RED = '#FF0000'
DK_RED = '#990000'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
LT_BLUE = '#0088FF'


In [2]:
# the ER Graph class

class ER(Graph):
        
    def set_vertex_list(self, q):
        # generates left-normalized vertices over GF(q)^3 in lexicographic order 
        self.q = q
        vertex_list = [Vertex([0,0,1])]
        for i in range(0,self.q):
            vertex_list.append(Vertex([0,1,i]))
        for i in range(0,self.q):
            for j in range(0,self.q):
                vertex_list.append(Vertex([1,i,j]))
        return vertex_list, len(vertex_list)
    
    def set_adjacency_matrix(self):
        # adjacency_matrix[i,j] is 1 if vertex_list[i].value is orthogonal to vertex_list[j].value and 0 otherwise
        self.gfq = GFq(self.q)
        num_vertices = len(self.vertex_list)
        adj_matrix = []
        for i in range(num_vertices):
            row = [0]*num_vertices
            adj_matrix.append(row)
        for i in range(num_vertices):
            d = self.gfq.dot_product_3d(self.vertex_list[i].value, self.vertex_list[i].value)
            if d==0:
                adj_matrix[i][i]=1
        for i in range(num_vertices):
            for j in range(i+1,num_vertices):
                d = self.gfq.dot_product_3d(self.vertex_list[i].value, self.vertex_list[j].value)
                if d==0:
                    adj_matrix[i][j]=1
                    adj_matrix[j][i]=1
        return adj_matrix
    
    def set_layouts(self):
        self.layouts = ER_Layouts(self)
    
    def init_stuff(self):
        # the next is specific to ER graphs
        self.L,self.W,self.C,self.V1,self.V2 = self.get_LWCV_subsets_ER()        
        if q<17:
            self.edges_list.list[0].opacity = 0.2
        elif q<27:
            self.edges_list.list[0].opacity = 0.08
        elif q<47:
            self.edges_list.list[0].opacity = 0.02
        elif q<63:
            self.edges_list.list[0].opacity = 0.01
        elif q<129:
            self.edges_list.list[0].opacity = 0.001
        else:
            self.edges_list.list[0].opacity = 0.0002
    
    def get_LWCV_subsets_ER(self):
        L_color = RED
        W_color = DK_RED
        C_color = LT_GREEN
        V1_color = DK_GREEN
        V2_color = LT_BLUE
        L=[]
        W=[]
        # make W
        for i in range(self.num_vertices):
            if self.adj_matrix[i][i]==1:
                W.append(i)
                self.vertex_list[i].color = W_color
        # grab the starter quadric L out of W: the first element of W
        L.append(W[0])
        self.vertex_list[W[0]].color = L_color
        W.pop(0)
        C=[]
        V1=[]
        for i in range(self.num_vertices):
            if (i not in L) and (i not in W) and self.adj_matrix[L[0]][i]==1:
                C.append(i)   
                self.vertex_list[i].color = C_color
        for i in range(self.num_vertices):
            if (i not in L) and (i not in W) and (i not in C):
                for j in W:
                    if self.adj_matrix[i][j]==1:
                        V1.append(i)
                        self.vertex_list[i].color = V1_color
                        break  # only need to be orthogonal to one member of W
        V2 = list(set(range(self.num_vertices)) - set(L+W+C+V1))
        for i in V2:
            self.vertex_list[i].color = V2_color
        return (L, W, C, V1, V2)

    def get_cluster_ER(self, c):
    # get the cluster generated from a given center element
        clusterptrs=[c]
        for i in range(self.num_vertices):
            if self.adj_matrix[c][i]==1 and self.adj_matrix[i][i]==0:
                clusterptrs.append(i)
        return clusterptrs
    
    def get_all_clusters_ER(self):
    # get the cluster generated from a given quadric and center element
        clusters=[]
        for i in range(q):
            cluster=[self.C[i]]
            for j in range(self.num_vertices):
                if self.adj_matrix[self.C[i]][j]==1 and self.adj_matrix[j][j]==0:
                    cluster.append(j)
            clusters.append(cluster)
        return clusters
    
    def set_interesting_edgelists_ER(self):
        cluster_main = self.get_cluster_ER(self.C[0])
        ee = self.get_edges_within_indexset(cluster_main, 'black', 1.0, 3)
        self.edges_list.list.append(self.get_edges_within_indexset(cluster_main, 'black', 1.0, 3))
        self.edges_list.names.append('cluster')

        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, self.W+self.L, 'red', 1.0, 3))
        self.edges_list.names.append('cluster_to_quadrics')

        cluster_nbr = self.get_cluster_ER(self.C[len(self.C)-1])
        self.edges_list.list.append(self.get_edges_within_indexset(cluster_nbr, 'black', 1.0, 3)) 
        self.edges_list.names.append('cluster_nbr')

        cluster_nbr_V1 = list(set(cluster_nbr) & set(self.V1))
        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, cluster_nbr_V1, LT_GREEN, 1.0, 3)) 
        self.edges_list.names.append('cluster_to_nbrV1')

        cluster_nbr_V2 = list(set(cluster_nbr) & set(self.V2))
        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, cluster_nbr_V2, LT_BLUE, 1.0, 3)) 
        self.edges_list.names.append('cluster_to_nbrV2')

        clusters_all=[]
        es = Edgeset('black', 1.0, 2)
        for i in range(len(self.C)):
            clusters_all.append(self.get_cluster_ER(self.C[i]) )
            cluster_es = self.get_edges_within_indexset(clusters_all[i], 'black', 1.0, 3)
            es.concatenate_edgesets(cluster_es.edges)
        self.edges_list.list.append(es)
        self.edges_list.names.append('all_clusters')
        
        return self.edges_list, self.edges_list.names



In [3]:
class ER_Layouts(Layouts):
    
    def set_layouts_ER(self): 
    # in addition to the clockwise default
        self.add_layout(self.good_layout_ER(), 'good') 
        self.add_layout(self.cake_layout_ER(), 'cake')
        return self.layout_list, self.layout_names
            
    def good_layout_ER(self):
    # this method permutes the positions of the vertices in a way that helps to show how clusters are made
        # copy the good layout: cake_layout is based on good_layout
        vpos_list = self.graph.num_vertices*[[0,0,0]]
        floor_q_half = int((self.graph.q-1)/2)
        ceil_q_half = int((self.graph.q+1)/2)
        angle_increment = 2*np.pi/self.graph.num_vertices    
        
        # L: place it (q+1)/2 to the left of the top
        vpos_list[self.graph.L[0]] = self.circle_pos(self.graph.num_vertices-ceil_q_half)
        # C: place these spread over the top of the circle
        for i in range(self.graph.q):
            vpos_list[self.graph.C[i]] = self.circle_pos((i - floor_q_half)%self.graph.num_vertices)
        current_pos = ceil_q_half        
        # W, V1, V2
        count = 1
        for i in range(self.graph.q):      
            # W: interspersed between each "rack" C_i of V1 and V2 points
            if i == floor_q_half:    # put this one opposite L[0]
                cir_pos = ceil_q_half
            else:  # the rest are corresponding to C[i]
                cir_pos = self.graph.num_vertices-ceil_q_half-count*self.graph.q
                count += 1
            for j in range(self.graph.q):
                if self.graph.C[i] in self.graph.vertex_list[self.graph.W[j]].adj:
                    vpos_list[self.graph.W[j]] = self.circle_pos(cir_pos)                
            # V1 and V2: to points corresponding to their cluster centers from C
            V1_Ci=[]
            V2_Ci=[]
            for j in range(len(self.graph.V1)):
                if self.graph.adj_matrix[self.graph.V1[j]][self.graph.C[q-1-i]]==1:
                    V1_Ci.append(self.graph.V1[j])
                if self.graph.adj_matrix[self.graph.V2[j]][self.graph.C[q-1-i]]==1:
                    V2_Ci.append(self.graph.V2[j])
            current_pos += 1
            if (self.graph.q%4 == 3): 
                for j in range(floor_q_half):
                    # move the jth V1_Ci
                    vpos_list[V1_Ci[j]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #  and match it up with the V2 that is orthogonal
                    for k in range(floor_q_half):
                        if V2_Ci[k] in self.graph.vertex_list[V1_Ci[j]].adj:
                            vpos_list[V2_Ci[k]] = self.circle_pos(current_pos)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther
            else:   # q%4 == 1
                # we'll move through the elts of V1 and V2, popping the pair every time we find a twinned match
                for j in range(int((self.graph.q-1)/4)): 
                    # pick the first V1 and V2 (any one not picked will do), alternate these
                    vpos_list[V1_Ci[0]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #pick twins of V1_temp[0]
                    for k in range(1,floor_q_half):
                        if V1_Ci[k] in self.graph.vertex_list[V1_Ci[0]].adj:
                            vpos_list[V1_Ci[k]] = self.circle_pos(current_pos)
                            V1_Ci.pop(k)
                            V1_Ci.pop(0)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther
                    vpos_list[V2_Ci[0]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #pick twins of V2_temp[0]
                    for k in range(1,floor_q_half):
                        if V2_Ci[k] in self.graph.vertex_list[V2_Ci[0]].adj:
                            vpos_list[V2_Ci[k]] = self.circle_pos(current_pos)
                            V2_Ci.pop(k)
                            V2_Ci.pop(0)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther               
        return vpos_list

    def cake_layout_ER(self):
    # copy the good layout: cake_layout is based on good_layout
        vpos_list = []
        for i in range(self.graph.num_vertices):
            vpos_list.append(copy.deepcopy(self.graph.vertex_list[i].pos[1]) )      
    # L,W: move away from the viewer and into the center, starting_angle=0, height=-1
        LW = self.graph.L+self.graph.W
        radius_W = .25
        LW_vpos_list = self.all_circle_positions(LW, 0, radius_W, -1.0)
        vpos_list[self.graph.L[0]] = LW_vpos_list[0]
        for i in range(self.graph.q):
            vpos_list[self.graph.W[i]] = LW_vpos_list[i+1]
    # C: Move into the center, starting_angle=-np.pi/q, height=-0
        radius_C= 1.5*radius_W
        C_vpos_list = self.all_circle_positions(self.graph.C, -np.pi/q, radius_C, 0)
        for i in range(self.graph.q):
            vpos_list[self.graph.C[i]] = C_vpos_list[i]
    # V1: leave them where they are in the good layout
    # V2: Move the toward the viewer at positive z
        for i in range(len(self.graph.V2)):        
            vpos_list[self.graph.V2[i]][2] += 1.0
    # This permutation on the quadrics is used in the paper (Fig. 7) to cleanly show quadrics connections for q=7        
        if self.graph.q == 7:
            for i in range(self.graph.q+1):
                LW_vpos_list[i] = vpos_list[LW[i]]
            new_LW_vpos_list = self.permute_pos(LW_vpos_list, [[0,1,2,4,6,7]])
            for i in range(self.graph.q+1):
                vpos_list[LW[i]] = new_LW_vpos_list[i]
        return vpos_list


In [7]:
#####    these are the prime powers q to be run         ##### 
#####    this algorithm is for ODD PRIME POWERS only    ##### 

# Here are some smaller primes
#prime_power_list = [3,5,7,9,11,13,17,19,23,25,27,29,31,37,41,43,47,49,53,59,61]

# These primes are of interest.
# Graphs for these primes support [556,993,2257,3783,16257] routers that have radixes [24,32,48,62,128]. 
# May want to change the opacity on edges and maybe the line width in the viz.
#prime_power_list = [23,31,47,61,127]  

#prime_power_list = [3,7]  # used for the paper viz

prime_power_list = [7,9]

for q in prime_power_list:
    ER_Graph = ER(q)
    layout_list, layout_names = ER_Graph.layouts.set_layouts_ER()
    edgelists,edgelist_names = ER_Graph.set_interesting_edgelists_ER()
    GF = galois.GF(q)

    ER_Subgraph = ER_Graph.induced_subgraph(ER_Graph.V1)
    
    visualize_graph(ER_Subgraph, 'good', 
                    ['all_edges'], 
                     'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_V1_subgraph", 
                    'GF('+str(q)+'): a subgraph, good layout')
           
# have a look at the basic layouts

# visualize_graph(graph, layout, edge_displaylist, camera_pos, camera_rot, show_labels, fname, caption)
# layouts: ['default', 'good', 'cake']
# edgelists: ['all_edges', 'cluster', 'cluster_to_quadrics', 'cluster_nbr', 'cluster_to_nbrV1', 'cluster_to_nbrV2', 'all_clusters']

    visualize_graph(ER_Graph, 'default', 
                    ['all_edges'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_lex", 
                    'GF('+str(q)+'): Lexicographic layout')
    visualize_graph(ER_Graph, 'good', 
                    ['all_edges'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_good", 
                    'GF('+str(q)+'): Good layout')
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges'], 
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_all", 
#                     'GF('+str(q)+'): Layer cake, all edges')    
# #   look at one cluster, cake layout
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges', 'cluster'],
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_1cluster", 
#                     'GF('+str(q)+'): Layer cake, one cluster')    
# #   look at the same cluster, plus its connections to the quadrics, cake layout
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges', 'cluster', 'cluster_to_quadrics'],
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_1cluster_quads", 
#                     'GF('+str(q)+'): Layer cake, one cluster plus quadrics')    
#   look at the same cluster, plus its connections to the quadrics and to a neighbor cluster, cake layout
    visualize_graph(ER_Graph, 'cake', 
                    ['all_edges', 'cluster', 'cluster_to_quadrics', 'cluster_nbr', 'cluster_to_nbrV1', 'cluster_to_nbrV2'],
                    'zx', [0,0,20], False,
                     "./ER_graphs/ER" + str(q) + "_cake_2clusters_edges", 
                    'GF('+str(q)+'): Layer cake, two clusters plus external edges')   
#   look at all clusters from overhead, cake layout
    visualize_graph(ER_Graph, 'cake', 
                    ['all_edges', 'all_clusters'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_cake_allclusters_overhead", 
                    'GF('+str(q)+'): Layer cake, all clusters, overhead')
    


GF(7): a subgraph, good layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(7): Lexicographic layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(7): Good layout


...

ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(9): Layer cake, two clusters plus external edges


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(9): Layer cake, all clusters, overhead


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)